This notebook contains the developent of a custom component for home-assistant for monitoring Hue motion sensors and 4 button remotes. The component is a RESTful sensor. It is split into 2 main sections: functions to parse the json data returned by the API call, and classes to represent the sensor data in HASS.

In [15]:
import requests
import json
import pprint
from datetime import datetime, timedelta

def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

# for loading URL    
def load_url(filename):
    try:
        with open(filename, 'r') as fp:
            url = json.load(fp)
    except Exception as e:
        print('Failed to load url')
        url = None
    return url['url']

In [16]:
filename = '/Users/robin/Desktop/Hue_url.json'

In [17]:
URL = load_url(filename)    # base URL of the remote, ending /sensors
sensors = requests.get(URL).json()

In [22]:
sensors

{'1': {'state': {'daylight': True, 'lastupdated': '2020-02-06T08:01:00'},
  'config': {'on': True,
   'configured': True,
   'sunriseoffset': 30,
   'sunsetoffset': -30},
  'name': 'Daylight',
  'type': 'Daylight',
  'modelid': 'PHDL00',
  'manufacturername': 'Philips',
  'swversion': '1.0'},
 '7': {'state': {'status': 0, 'lastupdated': '2019-04-21T05:23:00'},
  'config': {'on': True, 'reachable': True},
  'name': 'MotionSensor 5.Companion',
  'type': 'CLIPGenericStatus',
  'modelid': 'PHA_STATE',
  'manufacturername': 'Philips',
  'swversion': '1.0',
  'uniqueid': 'MotionSensor 5.Companion',
  'recycle': True},
 '12': {'state': {'presence': False, 'lastupdated': '2020-02-05T20:46:58'},
  'config': {'on': True, 'reachable': True},
  'name': 'Robins iPhone',
  'type': 'Geofence',
  'modelid': 'HA_GEOFENCE',
  'manufacturername': 'Philips',
  'swversion': 'A_1',
  'uniqueid': 'L_02_IYwHM',
  'recycle': False},
 '13': {'state': {'presence': False, 'lastupdated': '2020-02-05T20:46:58'},
  

In [18]:
sensors.keys()

dict_keys(['1', '7', '12', '13', '26', '27', '28', '32', '33', '34', '35', '36', '37', '38', '39', '44', '45', '66', '67', '68', '69', '70', '71', '73', '74', '75', '81', '82', '85', '86', '87', '88'])

Print out all my sensors

In [19]:
id_list = []
for i in range(1,25):
    try:
        type = sensors[str(i)]['type'][3:]   # First 3 characters always ZZL
        name = sensors[str(i)]['name']
        modelid = sensors[str(i)]['modelid'][0:3]
        id = modelid + '_' + sensors[str(i)]['uniqueid'].split(':')[-1][0:5]
        print(str(i) + ' : ' + type + ' : ' + modelid + ' : ' + name + ' : ' + id)
    except:
        pass

7 : PGenericStatus : PHA : MotionSensor 5.Companion : PHA_Motio
12 : fence : HA_ : Robins iPhone : HA__L_02_
13 : PPresence : HOM : HomeAway : HOM_L_01_


Individual devices indicated by first 5 characters in last entry in id_unique. 
I have 3 motion sensors and 2 remotes.

Will ignore scenes, with modelid PHWA01.

Look at the data for a remote.

In [20]:
remote_index = 12
remote_url = URL + '/{}'.format(remote_index)
remote = requests.get(remote_url).json()

In [21]:
print_json(remote)

{'config': {'on': True, 'reachable': True},
 'manufacturername': 'Philips',
 'modelid': 'HA_GEOFENCE',
 'name': 'Robins iPhone',
 'recycle': False,
 'state': {'lastupdated': '2020-02-05T20:46:58', 'presence': False},
 'swversion': 'A_1',
 'type': 'Geofence',
 'uniqueid': 'L_02_IYwHM'}


In [7]:
remote['state']#['buttonevent']

{'lastupdated': '2017-10-06T19:20:28', 'presence': True}

Format the time correctly

In [8]:
lastupdated = remote['state']['lastupdated']
lastupdated

'2017-10-06T19:20:28'

In [9]:
lastupdated.split('T')

['2017-10-06', '19:20:28']

Format the name correctly

In [10]:
line='Hall Sensor'
arr = line.split()
arr.insert(1, ' Motion ')
print(' '.join(arr))

Hall  Motion  Sensor


## Parse the 3 kinds of sensor
Functions to parse 3 kinds of sensor, and a function to parse the API json data returned which calls the 3 functions.
#### Parsing the SML001 Hue motion sensor

In [11]:
def parse_sml(response):
    """Parse the json for a SML Hue motion sensor and return the data."""
    if 'ambient light' in response['name']:
        lightlevel = response['state']['lightlevel']
        lux = round(float(10**((lightlevel-1)/10000)), 2)
        dark = response['state']['dark']
        daylight = response['state']['daylight']
        data = {'light_level': lightlevel,
                'lux': lux,
                'dark': dark,
                'daylight': daylight, }

    elif 'temperature' in response['name']:
        data = {'temperature': response['state']['temperature']/100.0}

    else:
        name_raw = response['name']
        arr = name_raw.split()
        arr.insert(-1, 'motion')
        name = ' '.join(arr)
        hue_state = response['state']['presence']
        if hue_state is True:
            state = 'on'
        else:
            state = 'off'

        data = {'model': 'Motion',
                'state': state,
                'battery': response['config']['battery'],
                'name': name}
    return data


In [12]:
motion_sensor = requests.get(URL + '/{}'.format(15)).json()
parse_sml(motion_sensor)

{'battery': 100,
 'model': 'Motion',
 'name': 'Living room motion sensor',
 'state': 'on'}

In [13]:
motion_sensor

{'config': {'alert': 'none',
  'battery': 100,
  'ledindication': False,
  'on': True,
  'pending': [],
  'reachable': True,
  'sensitivity': 2,
  'sensitivitymax': 2,
  'usertest': False},
 'manufacturername': 'Philips',
 'modelid': 'SML001',
 'name': 'Living room sensor',
 'state': {'lastupdated': '2017-10-07T07:06:37', 'presence': True},
 'swupdate': {'lastinstall': None, 'state': 'noupdates'},
 'swversion': '6.1.0.18912',
 'type': 'ZLLPresence',
 'uniqueid': '00:17:88:01:02:01:aa:9c-02-0406'}

In [14]:
motion_sensor = requests.get(URL + '/{}'.format(14)).json()
parse_sml(motion_sensor)

{'temperature': 21.52}

In [15]:
motion_sensor = requests.get(URL + '/{}'.format(16)).json()
parse_sml(motion_sensor)

{'dark': True, 'daylight': False, 'light_level': 6334, 'lux': 4.3}

#### Parsing the RWL021 Hue remote

In [16]:
def parse_rwl(response):
    """Parse the json response for a RWL Hue remote."""
    press = str(response['state']['buttonevent'])

    if press[-1] in ['0', '2']:
        button = str(press)[0] + '_click'
    else:
        button = str(press)[0] + '_hold'

    data = {'model': 'Remote',
            'name': response['name'],
            'state': button,
            'battery': response['config']['battery'],
            'last_updated': response['state']['lastupdated'].split('T')}
    return data

Removed this code for now, it checks if it has been greater than 5 seconds since last press of remote and if so sets the status to idle. However there is currently an issue that the Hue time is out by 1 hour so this doesn't work. Check why Hue time is out and add later if required.

    # Check how long since button pressed. If > seconds consider remote idle
    lastupdated = response['state']['lastupdated'].split('T')  # returns a list
    lastupdated_str = lastupdated[0] + ' ' + lastupdated[1]   # return a string
    lastupdated_obj = datetime.strptime(lastupdated_str, '%Y-%m-%d %H:%M:%S')  # convert to datetime object
    diff = datetime.now() - lastupdated_obj
    sec_since_update = diff.seconds

    # Why is API request returning wrong time zone? 
    #if diff.seconds > 5:
    #    button = 'idle'

In [17]:
remote = requests.get(URL + '/{}'.format(20)).json()
parse_rwl(remote)

{'battery': 100,
 'last_updated': ['2017-10-07', '06:30:28'],
 'model': 'Remote',
 'name': 'Living room remote',
 'state': '2_click'}

#### Parse the GeoFence
Hue presnce status

In [18]:
def parse_geofence(response):
    """Parse the json response for a GEOFENCE and return the data."""
    hue_state = response['state']['presence']
    if hue_state is True:
        state = 'on'
    else:
        state = 'off'
    data = {'name': response['name'],
            'model': 'Geofence',
            'state': state}
    return data

In [19]:
parse_geofence(requests.get(URL + '/{}'.format(12)).json())

{'model': 'Geofence', 'name': 'Robins iPhone', 'state': 'on'}

## Parsing the json file
Function to parse the entire json returned by the API call. Loop over all sensors and if any of the 3 sensors of interest is found, return their data

In [67]:
def parse_hue_api_response(response):
    """Take in the Hue API json response."""
    data_dict = {}    # The list of sensors, referenced by their hue_id.

    # Loop over all keys (1,2 etc) to identify sensors and get data.
    for key in response.keys():
        sensor = response[key]

        modelid = sensor['modelid'][0:3]
        if modelid in ['RWL', 'SML', 'ZGP']:
            _key = modelid + '_' + sensor['uniqueid'].split(':')[-1][0:5]

            if sensor['modelid'][0:3] == 'RWL':
                data_dict[_key] = parse_rwl(sensor)
            elif sensor['modelid'] == 'ZGPSWITCH':
                data_dict[_key] = parse_zpg(sensor)
            elif sensor['modelid'][0:3] == 'SML':
                if _key not in data_dict.keys():
                    data_dict[_key] = parse_sml(sensor)
                else:
                    data_dict[_key].update(parse_sml(sensor))

        elif sensor['modelid'] == 'HA_GEOFENCE':
            data_dict['Geofence'] = parse_geofence(sensor)
    return data_dict

In [68]:
my_parsed_data = parse_hue_api_response(requests.get(URL).json())
my_parsed_data

{'Geofence': {'model': 'Geofence', 'name': 'Robins iPhone', 'state': 'on'},
 'RWL_1e-02': {'battery': 100,
  'last_updated': ['2017-10-07', '06:30:28'],
  'model': 'Remote',
  'name': 'Living room remote',
  'state': '2_click'},
 'RWL_9c-02': {'battery': 100,
  'last_updated': ['2017-10-04', '06:04:54'],
  'model': 'Remote',
  'name': 'Hall remote',
  'state': '1_hold'},
 'RWL_dc-02': {'battery': 100,
  'last_updated': ['2017-10-07', '07:08:37'],
  'model': 'Remote',
  'name': 'Remote bedroom',
  'state': '4_click'},
 'SML_28-02': {'battery': 100,
  'dark': True,
  'daylight': False,
  'light_level': 6858,
  'lux': 4.85,
  'model': 'Motion',
  'name': 'Hall motion Sensor',
  'state': 'off',
  'temperature': 18.0},
 'SML_9c-02': {'battery': 100,
  'dark': True,
  'daylight': False,
  'light_level': 7144,
  'lux': 5.18,
  'model': 'Motion',
  'name': 'Living room motion sensor',
  'state': 'off',
  'temperature': 21.38},
 'SML_ce-02': {'battery': 100,
  'dark': True,
  'daylight': False,

# HA specific development
## Data class
A class to hold the parsed data. In HA this is onject is throttled so it is only called once a second. Therfore only a single sensor object will update the data object but since the data object is shared by all sensors it will be up to date for all sensors. 

In [22]:
class HueSensorData(object):
    """Get the latest sensor data."""

    def __init__(self, url):
        """Initialize the object."""
        self.url = url
        self.data = None

    # Update only once in scan interval.
   # @Throttle(SCAN_INTERVAL)         # included in HA
    def update(self):
        """Get the latest data"""
        response = requests.get(self.url)
        if response.status_code != 200:
            _LOGGER.warning("Invalid response from API")
        else:
            self.data = parse_hue_api_response(response.json())

In [23]:
my_data = HueSensorData(URL)    # I have the URL saved in a file, but the HA component loads it from phue.conf
my_data.update()

In [24]:
my_data.data.keys()   # sensors are indexed by their Hue ID

dict_keys(['dc-02', '28-02', 'ce-02', 'Geofence', '9c-02', '1e-02'])

## Sensor class

Create a generic HueSensor class.

In [44]:
class HueSensor():  # Entity
    """Class to hold Hue Sensor basic info."""

    ICON = 'mdi:run-fast'

    def __init__(self, hue_id, data):
        self._hue_id = hue_id
        self._data = data    # data is in .data
        self._name = self._data.data[self._hue_id]['name']
        self._model = self._data.data[self._hue_id]['model']
        self._state = self._data.data[self._hue_id]['state']
        self._attributes = {}

    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def device_state_attributes(self):
        """Only motion sensors have attributes currently, but could extend."""
        return self._attributes

    def update(self):
        """Update the sensor."""
        self._data.update()
        self._state = self._data.data[self._hue_id]['state']
        if self._model == 'SML':
            self._attributes['light_level'] = self._data.data[
                self._hue_id]['light_level']
            self._attributes['temperature'] = self._data.data[
                self._hue_id]['temperature']
        elif self._model == 'RWL':
            self._attributes['last updated'] = self._data.data[
                self._hue_id]['last_updated']

### Test the motion sensor

In [59]:
my_data.data['9c-02']

{'battery': 100,
 'last_updated': ['2017-10-04', '06:04:54'],
 'model': 'Remote',
 'name': 'Hall remote',
 'state': '1_hold'}

In [46]:
test_hue_motion = HueSensor('ce-02', my_data)
test_hue_motion.update()
test_hue_motion._hue_id

'ce-02'

In [47]:
test_hue_motion.name

'Bedroom motion sensor'

In [48]:
test_hue_motion.state

'off'

In [49]:
test_hue_motion.device_state_attributes

{}

### Test the remote

In [50]:
my_data.data['1e-02']

{'battery': 100,
 'last_updated': ['2017-10-07', '06:30:28'],
 'model': 'Remote',
 'name': 'Living room remote',
 'state': '2_click'}

In [51]:
test_hue_remote = HueSensor('1e-02', my_data)
test_hue_remote.update()
test_hue_remote._hue_id

'1e-02'

In [52]:
test_hue_remote.name

'Living room remote'

In [53]:
test_hue_remote.state

'2_click'

In [54]:
test_hue_remote.device_state_attributes

{}

### Test the Geofence

In [55]:
my_data.data['Geofence']

{'model': 'Geofence', 'name': 'Robins iPhone', 'state': 'on'}

In [56]:
test_geofence = HueSensor('Geofence', my_data)
test_geofence.update()
test_geofence._hue_id

'Geofence'

In [57]:
test_geofence.name

'Robins iPhone'

In [58]:
test_geofence.state

'on'

## Test setup platform
Mimic setup in HA

In [40]:
def setup_platform():
    """Set up the component."""
    data = HueSensorData(URL)
    data.update()
    sensors = []
    for key in data.data.keys():
        sensors.append(HueSensor(key, data))
    return(sensors)
    #add_devices(sensors, True)

In [41]:
my_sensors = setup_platform()

In [42]:
for sensor in my_sensors:
    sensor.update()
    print(sensor.name)
    print(sensor.state)
    print(sensor.device_state_attributes)
    print('*****')

Remote bedroom
4_click
{}
*****
Hall motion Sensor
off
{}
*****
Bedroom motion sensor
off
{}
*****
Robins iPhone
on
{}
*****
Hall remote
1_hold
{}
*****
Living room remote
2_click
{}
*****


## Testing
Use mocker to return a file rather than fresh API data, allows us to create tests with standardised results.

In [43]:
import requests_mock
import unittest

ModuleNotFoundError: No module named 'requests_mock'

In [ ]:
class TestHueSensor(unittest.TestCase):
    
    def test_hue_sensors(self):

        with requests_mock.mock() as m:
            """Test for operational uk_transport sensor with proper attributes."""
            mock_url  = 'http://mock'
            m.get(mock_url, text=open('tests//hue_sensors.json').read())           
            data = HueSensorData(mock_url)
            data.update()
            sensors = []
            for key in data.data.keys():
                sensor = HueSensor(key, data)
                sensor.update()
                sensors.append(sensor)
              
            assert len(sensors) == 6
            for sensor in sensors:
                if sensor.name == 'Living room motion sensor':
                    assert sensor.state is 'off'
                    assert sensor.device_state_attributes['light_level'] == 0
                    assert sensor.device_state_attributes['temperature'] == 21.38
                elif sensor.name == 'Living room remote':
                    #print(sensor.device_state_attributes['last updated'])
                    assert sensor.state == '1_hold'
                    assert sensor.device_state_attributes['last updated'] == ['2017-09-15', '16:35:00']
                elif sensor.name == 'Robins iPhone':
                    assert sensor.state is 'on'
                else:
                    assert sensor.name in ['Bedroom motion sensor', 'Remote bedroom', 'Hall motion Sensor'] 


unittest.main(argv=['ignored', '-v'], exit=False)

In [ ]:
a = {'key1':{'key2':'foo'}}

In [ ]:
iter_obj = a.keys().__iter__()

In [ ]:
next(iter_obj)

In [ ]:
next(iter_obj)

In [ ]:
next(json.loads(''.join(inp)).keys().__iter__())